Docs

In [ ]:
from genericpath import exists
from ezmodel.ezmetadataentry import EzMetadataEntry
from ezmodel.ezmetadatamodel import EzMetadataModel
from parsers.ang import parse_header_as_dict

from ht_requests.ht_requests import ht_utilities
from ht_requests.ht_requests import htauthcontroller
from ht_requests.ht_requests import ht_requests

import json
from typing import List



## Loading and Existing MetaForge Template ##

We are going to use an existing EBSD 2 Phase template located in the "example_templates" folder and load that into memory using the "from_json_file()" function.

In [ ]:


# Set teh path of the file to store the JSON of the model
ez_file_path = 'example_templates/MultiPhase.ez'

# Read the EzMetadataModel from the json file
model = EzMetadataModel.from_json_file(ez_file_path)


## Loading an ANG Dictionary from an existing Data file ##

We now want to load a data file whose values will be extracted and used as the meta data that gets uploaded and tagged onto the uploaded data file. 

In [ ]:
# Set the path of the data file to use to build the EzMetadataModel
data_file_path = '/Volumes/970-1/Data/Ang_Data/12_strain/Scan_10074.ang'

# Convert the ANG file into a dictionary
ang_dict = parse_header_as_dict(data_file_path)

Now we might want to sync the template with the actual ANG data that just got read from the .ang file. This can be important if you want to error out because template values were defined in the template but are *not* available in the actual data file. This can happen for instance if the template extracts data from multiple phases but the data file only has a single phase. In this code we simply print what is missing but proceed on. In real life you would probably want to error check or return early.

In [ ]:
# Sync the the EzMetadataModel from the ANG header dictionary
missing_entries: List[EzMetadataEntry] = model.update_model_values_from_dict(ang_dict)
if len(missing_entries) != 0:
  print('Not all values that appear in the Template file were in the input data file.')
  for e in missing_entries:
    print(f'{e.source_path}')

## Interacting with HyperThought ##

Now that we have setup our in memory template and verified our model is correct, we need to authenticate to HyperThought. We do this by creating an **auth_control** variable through the use of the *htauthcontroller* class.

**NOTE TO THE USER**
The access key used here is stale. You will need to go get a new Access Key from your HyperThought website.

In [ ]:
# Use the template to upload to HyperThought
# Set your API Access Key which you would get from HyperThought Web site
accessKey = 'eyJhY2Nlc3NUb2tlbiI6ICIzZGIyYzI1YjQ4NzE0NTMwODRkN2JjN2IyNzM4ZTAzMyIsICJyZWZyZXNoVG9rZW4iOiAiYzU2YzFjYjhiNzcwNGEyYTgwOWIxYjM0OTQyNTU1YzkiLCAiZXhwaXJlc0luIjogMzU5OSwgImV4cGlyZXNBdCI6ICIyMDIxLTA2LTI4VDE0OjE5OjU0LjY4NTAyMC0wNDowMCIsICJiYXNlVXJsIjogImh0dHBzOi8vaHQuYmx1ZXF1YXJ0ei5uZXQiLCAiY2xpZW50SWQiOiAiMDg3NzYwIiwgImNsaWVudFNlY3JldCI6ICIyYzMyYWJmMjAwZWRhNzE5MTQ0MTNmMmExMDUxOWJiNGMwMzFmZmI4MTk2MDQ0OTk1ZTg4MTllYyJ9'

# Create an HtAuthController to hold the API key
auth_control = htauthcontroller.HTAuthorizationController(accessKey)

# Upload to a folder created at the root level.
path = ","

# Set the remote directory to create. This DOES NOT check if that folder already exists
remoteDirPath = "Unit_Test"


# Get a list of the projects that the user has access to.
projects_list = ht_requests.list_projects(auth_control)


In [ ]:

# Print the project names {Optional}
for pj in projects_list:
    print(f'{pj["content"]["title"]}')

In [ ]:
# Look for a specific project by *Project Title*
project_title = 'Project X-Caliber'
project_exists = False
project_json = {}
project_pk = ""
for pj in projects_list:
    if pj["content"]["title"] == project_title:
        project_exists = True
        project_json = pj
        project_pk = project_json["content"]["pk"]

# Check to make sure we found the Project. In REAL LIFE you would probably error out at this point if the
# project was not found.
if project_exists == False:
    print(f'The requested project "{project_title}" does not exist')


In [ ]:
# Get the file/folder list from the project listed in the previous code
project_folder_list = ht_requests.get_item_dict_from_ht_path(auth_control, 
              ht_path='/',
              ht_space = 'project',
              ht_space_id=project_pk)

# Print the list of folders/files inside the project of interest [Optional]
for pf in project_folder_list:
    print(pf)



# folderlist = ht_requests._list_location_contents(auth_control, ht_id_path = path)
# for j in folderlist:
#   print(f'{j["content"]}')



In [ ]:
# Set the remote directory to create.
remote_exists = False
remoteDirPath = "Unit_Test"
remote_folder_uuid = ""
print("Checking if remote folder {remoteDirPath} exists")
for f in project_folder_list:
    print(f)
    if f["Name"] is remoteDirPath:
        remote_exists = True
        remote_folder_uuid = f["UUID"]
        print(f'name: {f["Name"]}  UUID: {f["UUID"]}')
  
if not remote_exists:
    print("Remote Folder does not exist.. creating remote folder {remoteDirPath}")
    ht_id_path = ht_requests.get_ht_id_path_from_ht_path(auth_control, 
                        ht_path = '/',
                        ht_space = 'project',
                        ht_space_id=project_pk
                        )


    folder_id = ht_requests.create_folder(auth_control,
                folder_name = remoteDirPath,  
                ht_space = 'project',
                ht_space_id = project_pk,
                ht_id_path= ht_id_path)

    # Get the file/folder list from the project listed in the previous code [Optional]
    # This part is optional but probably necessary in real life to sanity check that the
    # requested directory was created.
    project_folder_list = ht_requests.get_item_dict_from_ht_path(auth_control, 
                ht_path='/',
                ht_space = 'project',
                ht_space_id=project_pk)

    # Print the list of folders/files inside the project of interest [Optional]
    for pf in project_folder_list:
        print(pf)


## Uploading Data Files with MetaData ##

We are going to upload a data file with Meta-Data extracted from a given Data file using a give template.

In [ ]:
# Extract the Meta-Data from the Template/Model
missing_entries = []
metadata = ht_utilities.ezmodel_to_ht_metadata(model=model, missing_entries=missing_entries)

# Pick your data files to upload
filelist = "/Users/mjackson/Downloads/011.ang"

# get the UUID of the new folder
folderuuid = ht_requests.get_ht_id_path_from_ht_path(auth_control, 
                                    ht_path=remoteDirPath, 
                                    ht_space = 'project', 
                                    ht_space_id=project_pk)
        

# Perform the upload.
file_id, file_name = ht_requests.upload_file(auth_control=auth_control, 
                                            local_path=filelist,
                                            ht_space='project',
                                            ht_space_id=project_pk,
                                            ht_id_path=folderuuid, 
                                            metadata=metadata)
print(f'{file_name}: {file_id}')
print("Upload completed")
